In [1]:
from pyspark.sql import SparkSession
import getpass
username = getpass.getuser()
spark = SparkSession. \
builder. \
config('spark.ui.port','0'). \
config("spark.sql.warehouse.dir", f"/user/itv014453/warehouse"). \
enableHiveSupport(). \
master('yarn'). \
getOrCreate()

In [2]:
#WritePySparkcodetocreateanewdataframewiththedatagivenbelowhaving2columns(‘season’)and(‘windspeed’).[Datatypesofthecolumnnamescanbeinferred] 
data = [("Spring",12.3),("Summer",10.5),("Autumn",8.2),("Winter",15.1)]
data_schema = ['season','windspeed']

In [3]:
data_df = spark.createDataFrame(data,data_schema).show()

+------+---------+
|season|windspeed|
+------+---------+
|Spring|     12.3|
|Summer|     10.5|
|Autumn|      8.2|
|Winter|     15.1|
+------+---------+



In [4]:
#Considerthelibrarymanagementdatasetlocatedatthefollowingpath(/public/trendytech/datasets/library_data.json).UsingPySpark,loadthedataintoaDataframeandenforceschemausingStructType.
!hadoop fs -head /public/trendytech/datasets/library_data.json

{"library_name": "Central Library","location": "City Center","books": [{"book_id": "B001","book_name": "The Great Gatsby","author": "F. Scott Fitzgerald","copies_available": 5},{"book_id": "B002","book_name": "To Kill a Mockingbird","author": "Harper Lee","copies_available": 3}],"members": [{"member_id": "M001","member_name": "John Smith","age": 28,"books_borrowed": ["B001"]},{"member_id": "M002","member_name": "Emma Johnson","age": 35,"books_borrowed": []}]},
{"library_name": "Community Library","location": "Suburb","books": [{"book_id": "B003","book_name": "1984","author": "George Orwell","copies_available": 2},{"book_id": "B004","book_name": "Pride and Prejudice","author": "Jane Austen","copies_available": 4}],"members": [{"member_id": "M003","member_name": "Michael Brown","age": 42,"books_borrowed": ["B003","B004"]},{"member_id": "M004","member_name": "Sophia Davis","age": 31,"books_borrowed": ["B004"]}]}


In [5]:
from pyspark.sql.types import *
library_schema = StructType([
    StructField('library_name',StringType()),
    StructField('location',StringType()),
    StructField('books',ArrayType(
        StructType([
            StructField('book_id',StringType()),
            StructField('book_name',StringType()),
            StructField('author',StringType()),
            StructField('copies_available',IntegerType())
        ])
    )),
    StructField('members',ArrayType(
        StructType([
            StructField('member_id',StringType()),
            StructField('member_name',StringType()),
            StructField('age',IntegerType()),
            StructField('books_borrowed',ArrayType(StringType()))
        ])
    ))
])

In [6]:
library_df = spark.read.schema(library_schema).json("/public/trendytech/datasets/library_data.json")

In [7]:
library_df.printSchema()

root
 |-- library_name: string (nullable = true)
 |-- location: string (nullable = true)
 |-- books: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- book_id: string (nullable = true)
 |    |    |-- book_name: string (nullable = true)
 |    |    |-- author: string (nullable = true)
 |    |    |-- copies_available: integer (nullable = true)
 |-- members: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- member_id: string (nullable = true)
 |    |    |-- member_name: string (nullable = true)
 |    |    |-- age: integer (nullable = true)
 |    |    |-- books_borrowed: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)



In [8]:
#Giventhedataset(/public/trendytech/datasets/train.csv),createaDataframeusingPySparkandperformthefollowingoperations
#a)Dropthecolumnspassenger_nameandagefromthedataset.
#b)Countthenumberofrowsafterremovingduplicatesofcolumnstrain_numberandticket_number.
#c)Countthenumberofuniquetrainnames

In [9]:
!hadoop fs -head /public/trendytech/datasets/train.csv

train_number,train_name,seats_available,passenger_name,age,ticket_number,seat_number
123,Express,100,John,25,T123,A1
123,Express,100,Emma,30,T124,B2
456,Superfast,150,Michael,35,T125,C3
456,Superfast,150,Sophia,40,T126,D4
789,Local,50,William,28,T127,E5
789,Local,50,Sophia,32,T128,F6
789,Local,50,Oliver,45,T129,G7


In [10]:
train_df = spark.read.csv("/public/trendytech/datasets/train.csv",inferSchema="true",header="true")

In [11]:
train_df.printSchema()

root
 |-- train_number: integer (nullable = true)
 |-- train_name: string (nullable = true)
 |-- seats_available: integer (nullable = true)
 |-- passenger_name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- ticket_number: string (nullable = true)
 |-- seat_number: string (nullable = true)



In [12]:
train_drop = train_df.drop("passenger_name","age")

In [13]:
train_drop.show()

+------------+----------+---------------+-------------+-----------+
|train_number|train_name|seats_available|ticket_number|seat_number|
+------------+----------+---------------+-------------+-----------+
|         123|   Express|            100|         T123|         A1|
|         123|   Express|            100|         T124|         B2|
|         456| Superfast|            150|         T125|         C3|
|         456| Superfast|            150|         T126|         D4|
|         789|     Local|             50|         T127|         E5|
|         789|     Local|             50|         T128|         F6|
|         789|     Local|             50|         T129|         G7|
+------------+----------+---------------+-------------+-----------+



In [14]:
train_drop_duplicates = train_drop.dropDuplicates(["train_number","ticket_number"])

In [15]:
train_drop_duplicates.count()

7

In [16]:
train_drop_duplicates.select("train_name").distinct().count()

3

##YouareworkingasaDataEngineerinalargeretailcompany.Thecompanyhasadatasetnamed"sales_data.json"thatcontainssalesrecordsfromvariousstores.
##ThedatasetisstoredinJSONformatandmayhavesomecorruptormalformedrecordsduetooccasionaldataqualityissues.
##Yourtaskistoreadthe"sales_data.json"dataset(/public/trendytech/datasets/sales_data.json)usingPySpark,utilizingdifferentreadmodestohandlecorruptrecords.
##YouneedtocreateaDataframeusingpysparkandperformthefollowingoperations:
##1.Readthedatasetusingthe"permissive"modeandcountthenumberofrecordsread.
##2.Readthedatasetusingthe"dropmalformed"modeanddisplaythenumberofmalformedrecords.
##3.Readthedatasetusingthe"failfast"mode.

!hadoop fs -head /public/trendytech/datasets/sales_data.json

In [26]:
schema = "store_id integer,product string,quantity integer,revenue double"
sales_data_df = spark.read.schema(schema).json("/public/trendytech/datasets/sales_data.json")

In [29]:
sales_data_df.count()

22

In [30]:
sales_data_df_2 = spark.read.schema(schema).option("mode","dropmalformed").json("/public/trendytech/datasets/sales_data.json")

In [31]:
sales_data_df_2.count()

21

In [35]:
sales_data_df_3 = spark.read.schema(schema).option("mode","failfast").json("/public/trendytech/datasets/sales_data.json")

In [ ]:
sales_data_df_3.show()